# Assignment 2: Practical Machine Learning Project Report



31005/32513 Machine Learning Spring 2019, Assignment 2



---



> Group members: Pit Wegner, Xianfeng Zhuge, Kailei Wu




## Introduction

Nowadays, more and more activity recognition-based machine learning has been studied, especially in human physical activity recognition. The progress in this direction is surprisingly good. Models can recognize simple human exercises, such as walking, running or standing, count steps and detect a general overarching activity. However, it is hard to apply in a specific domain such as healthcare. There are mainly for two reasons: the activities themselves are more complex, potentially involving other subjects, and additionally harder to analyze because of the lack of public data that can be used within the research. In this report, we create a machine learning (ML) model regarding the "Open Lab Nursing Activity Recognition Challenge", which asks to recognize six different activities within the nursing area (Lago et al., 2019).

In recent years, many activity recognition algorithms in health care applications are focusing on patients or doctors rather than nurses. However, the recognition of nursing activities can be helpful in the nursing domain, such as for automatic record creation and standardization of operation supervision. Until now, there is still a gap for ML models to recognize nursing-related activities due to the problems that we mentioned before. The applications of such a model would be wide-spread and of high impact in the nursing area, given the model achieves an exceptional accuracy in prediction.

In this report we would explain in detail about the data that we used including data preprocessing and data structure design, CNN model implementation that solved the problem, testing the performance of model and evaluation for whole experiment with testing result, efficiency analysis and the feature that we could improve our project in the future. 


## Exploration

The “Open Lab Nursing Activity Recognition Challenge” provides several sets of data for training propose, we selected the data that including 

The data that we used for this experiment is 

1. from the paper
2. data from the challenge, we are using motion capture data (sigment) and activitiy label 
3. data preprocessing, data split, data sampling (random pick windows, no doulbe for each epoch) 
4. CNN model, 1 conv, 1 pool, 2 fully connected, activation function on conv and fully connected1 (with graph) lossing(cross entropy loss) optimizer (adam (adaptive moment extimation))

5. test: normal test round, then sigment voting 




![Architecture](https://github.com/pitwegner/UTS_ML2019_Project/blob/master/img/architecture.svg)

## Methodology 

training: 

1. input the data 
2. mini batch: size 32
3. learning rate 0.000001
4. forward pass, backward pass, optimize
5. at the end of each epoch, we did validation(see the validation loss)
6. stop learning if valodation loss is not decrease for 10 epoch

testing: 

1. forward pass for test data set.
2. conpare to the labels, create confusion matrix, calculate the accuracy. 
3. majority vote for siugments, select the mode, calculate the accuracy.




## Evaluation

![Person_Split](https://github.com/pitwegner/UTS_ML2019_Project/blob/master/img/plot_person_split.png)
![Random_Segment_Split](https://github.com/pitwegner/UTS_ML2019_Project/blob/master/img/plot_random_segment_split.png)

## Conclusion

## Ethical Considerations

As a machine learning model, common ethical issues considering the application in a real world domain arise naturally. These typically aim at potential mistakes, training bias and data protection. Escpecially in the medical domain, mistakes can have a huge impact on a human life. In the case of our model, however, a wrong documentation entry about a nurse activity has a limited capability to directly cause damage. Patient health can be compromised by a left out or wrong procedure, for example developing pressure ulcers as a result of improper turning and repositioning of bedridden patients. The documentation record can then be used to trace back where the mistake has happened. Thus, our model provides support in preventing and retracing clinical incidents without directly impacting patient safety. Additionally, giving the nurse a chance to manually adjust the report after its generation further mitigates the impact of erroneous results. Other benefits of applying the model in nursing include a decreased workload and a more complete activity report for accounting purposes.

Activity recognition in a workplace also adds ethical complications. If not protected, tool could easily be (mis-)used by managers to monitor workforce activity, intruding into their privacy. The insights could result in different treatment for individual employees, even up to terminating the employment. Privacy impairment from the data pool itself is a further issue, since detailed movements are possibly visible from motion capture data, even if much harder to process than, say video. Thus, secure storage and access to data, as well as a proper authorization to the model results are crucial for an implementation.

Considering the results being biased toward training data, the risk of discrimination only leads to more inaccurate results for new employees. As this is known and can be tested beforehand, the impact is minimal. Overall, with proper data protection and sufficient accuracy, the benefits of the technique outweigh the possible risks. From a utalitarian point of view, the use of our model can hence be considered ethically acceptable. A deontological approach also does not object. Since results can be modified, the autonomy and freedom of will are not inhibited by applying the model, as long as privacy is ensured. Furthermore, the universalization of the application neither contradict with the intention nor with itself.